# DSCI-511 - Data preprocessing and acquisition

## Application to retrieve real time tweets on climate change.


### Team Members

Abhinav Chaudhary

Amar Kumar

Astha Jain

Erica Racine

## A)  Tweet Streamer - Connects to Twitter streaming API using tweepy and sends the tweet to rabbitmq queue.

In [ ]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import pika

class StreamTweets(StreamListener):
    def __init__(self,chan):
         self.channel = chan
    def on_data(self, tweet):
        try:
            self.channel.basic_publish(exchange='', routing_key='twitterqueue', body=tweet)
        except Exception as e:
            print("Error: %s" % e)

    def on_error(self,status_code):
        print(status_code)

    def on_exception(self, exception):
        print(exception)

if __name__ == "__main__":
        consumer_key = "ULItewxGYkR8ueTHxVpP7aVmE"
        consumer_secret = "tTYMqhqDs5hKQxvBfnz1KiUWaJKEq3PtXjniYvycdjmJteUYaQ"
        access_token = "1186025638622367745-Mb6BZ1vNn1dlOTgYs5MadKwO45M496"
        access_token_secret = "mntEfhh9o53i3m5jfSwRilCYOvGht42gby5eMOlSkzQ6r"

        auth = OAuthHandler(consumer_key,consumer_secret)
        auth.set_access_token(access_token,access_token_secret)

        connection = pika.BlockingConnection(
            pika.ConnectionParameters(host='localhost'))
        channel = connection.channel()

        channel.queue_declare(queue='twitterqueue')

        tracklist = ["#climatechange", "#climatechangeisreal", "#actonclimate", "#globalwarming",
                     "#climatechangehoax", "#climatedeniers", "#climatechangeisfalse", "#globalwarminghoax",
                     "#climatechangenotreal","#greentweets", "#carbondioxide" ,"#CO2", "#carbonemissions",
                     "#greenhousegas","#greenhousegasses","#greenhousegasemissions",'#climatechange','#nature',
                     '#environment', '#climate', '#climatestrike', '#globalwarming', '#savetheplanet', '#sustainability',
                     '#gretathunberg', '#zerowaste', '#ecofriendly', '#climatecrisis', '#fridaysforfuture', '#earth',
                     '#climateaction', '#climatechangeisreal', '#plasticfree', '#gogreen', '#savetheearth', '#sustainable',
                     '#climatejustice', '#pollution', '#recycle', '#green', '#eco', '#climateemergency', '#extinctionrebellion',
                     '#vegan', '#saveourplanet']

        stream = Stream(auth,listener = StreamTweets(channel))
        print("Starting twitter stream..")
        stream.filter(track = tracklist, languages = ["en"])




## B)  Tweet processor - Receives tweet from rabbitmq queue and processes the data before writing it to the CSV file.

In [ ]:
import pika
import json
import csv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
import os


def process_tweet(data):
    try:
            id = data["id"]
            text = data["text"].strip()
            created_at = data["created_at"]
            username = data["user"]["name"]
            user_id = data["user"]["id"]
            user_location = data["user"]["location"]
            quoted_full_text = None
            retweet_full_text = None
            if 'quoted_status' in data and 'extended_tweet' in data['quoted_status'].keys():
                     quoted_full_text = data['quoted_status']['extended_tweet']['full_text'].strip()
            elif 'retweeted_status' in data and 'extended_tweet' in data['retweeted_status'].keys():
                     retweet_full_text = data['retweeted_status']['extended_tweet']['full_text'].strip()
            ps = PorterStemmer()
            stop_words = set(stopwords.words('english'))
            if quoted_full_text is None and retweet_full_text is None:
                     httpremoved = re.sub(r"http\S+", "",text)
                     no_special_charac = re.sub('[^A-Za-z0-9\s]+', '', httpremoved)
                     words = word_tokenize(no_special_charac)
                     not_common_words = [w for w in words if not w in stop_words]
                     stemmed_words = [ps.stem(w) for w in not_common_words]
                     row = [id,created_at,username,user_id,user_location,text,stemmed_words]
            elif quoted_full_text is None:
                     httpremoved = re.sub(r"http\S+", "", retweet_full_text)
                     no_special_charac = re.sub('[^A-Za-z0-9\s]+', '', httpremoved)
                     words = word_tokenize(no_special_charac)
                     not_common_words = [w for w in words if not w in stop_words]
                     stemmed_words = [ps.stem(w) for w in not_common_words]
                     row = [id,created_at,username,user_id,user_location,retweet_full_text,stemmed_words]
            elif retweet_full_text is None:
                     httpremoved = re.sub(r"http\S+", "", quoted_full_text)
                     no_special_charac = re.sub('[^A-Za-z0-9\s]+', '', httpremoved)
                     words = word_tokenize(no_special_charac)
                     not_common_words = [w for w in words if not w in stop_words]
                     stemmed_words = [ps.stem(w) for w in not_common_words]
                     row = [id, created_at, username, user_id, user_location, quoted_full_text,stemmed_words]
            writer.writerow(row)
    except Exception as e:
            file.close()
            print("Error: %s" %e)

def callback(ch, method, properties, body):
    data = json.loads(body)
    process_tweet(data)


if __name__ == "__main__":
    header = ['tweet_id', 'created_at', 'username', 'user_id', 'user_location', 'text','stemmed_words']
    if not os.path.isfile('/home/tweet/tweet_data.csv'):
            file =  open("/home/tweet/tweet_data.csv", "a",encoding="utf-8")
            writer = csv.writer(file,delimiter=',')
            writer.writerow(header)
    else:
        file = open("/home/tweet/tweet_data.csv", "a", encoding="utf-8")
        writer = csv.writer(file, delimiter=',')
    connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))
    channel = connection.channel()
    channel.queue_declare(queue='twitterqueue')
    channel.basic_consume(
    queue='twitterqueue', on_message_callback=callback, auto_ack=True)
    print('Waiting for Tweets..')

    channel.start_consuming()

## C)  Sample Data

In [14]:
from IPython.display import display, HTML

import pandas as pd
df = pd.read_csv("tweet_data.csv")
display(HTML(df.sample(5).to_html()))

,tweet_id,created_at,username,user_id,user_location,text,stemmed_words
34544,1201157109074866177,Sun Dec 01 15:12:25 +0000 2019,John Snowmountainer,1003949688465354752,NaN,What are your options when it comes to replacing your boiler?\nhttps://t.co/Pvpn949j7h \n.\n#energy #energyefficiency #epc #renewableenergy #heating #cooling #environmentallyfriendly #energyefficient #homeowner #home #tips #savingmoney #sustainable #co2 #carbonfootprint,"['what', 'option', 'come', 'replac', 'boiler', 'energi', 'energyeffici', 'epc', 'renewableenergi', 'heat', 'cool', 'environmentallyfriendli', 'energyeffici', 'homeown', 'home', 'tip', 'savingmoney', 'sustain', 'co2', 'carbonfootprint']"
28508,1201139355378765825,Sun Dec 01 14:01:53 +0000 2019,Its @InnoDeckoks,1289740650,Nairobi,"The world again meets in Spain on to advance work on tackling #ClimateEmergency #ClimateCrisis Our #YOUTH message to #COP25Madrid is simple: science has shown we have no time, the people have shown we want #ClimateActionNow. Remember the people you serve and youth you'll leave! https://t.co/fHYp64PZnA","['the', 'world', 'meet', 'spain', 'advanc', 'work', 'tackl', 'climateemerg', 'climatecrisi', 'our', 'youth', 'messag', 'cop25madrid', 'simpl', 'scienc', 'shown', 'time', 'peopl', 'shown', 'want', 'climateactionnow', 'rememb', 'peopl', 'serv', 'youth', 'youll', 'leav']"
50683,1201204145971175425,Sun Dec 01 18:19:20 +0000 2019,DJMartin,1051231829754343424,NaN,"""It is heartbreaking for us that you have to come here and beg for your lives, that you have to come here and beg for the right to have a normal life"".\n\nhttps://t.co/10iiTsGcW6\n\n#SchoolStrike4Climate #YouthStrike4Climate #ExtinctionRebellion","['It', 'heartbreak', 'us', 'come', 'beg', 'live', 'come', 'beg', 'right', 'normal', 'life', 'schoolstrike4clim', 'youthstrike4clim', 'extinctionrebellion']"
42665,1201179935823605760,Sun Dec 01 16:43:08 +0000 2019,Kore Strong,960564431833845760,NaN,"""Bondar questioned how the three trustees got elected to sit on a board of education, given their denial of recognized science.""\n\nThe thought police are coming after school board trustees now. It's total indoctrination. \n#ClimateChangeHoax \n#bcpoli \nhttps://t.co/M1fUzJIqqM","['bondar', 'question', 'three', 'truste', 'got', 'elect', 'sit', 'board', 'educ', 'given', 'denial', 'recogn', 'scienc', 'the', 'thought', 'polic', 'come', 'school', 'board', 'truste', 'it', 'total', 'indoctrin', 'climatechangehoax', 'bcpoli']"
38796,1201169098681585665,Sun Dec 01 16:00:04 +0000 2019,Digital Sages,859204328430678016,"New Jersey, USA",RT @aTravelCompanio: #London tops list of world’s most #vegan friendly cities\nhttps://t.co/9VIaxcPcM1 #foodies https://t.co/ouRkhqk6BA,"['RT', 'atravelcompanio', 'london', 'top', 'list', 'world', 'vegan', 'friendli', 'citi', 'foodi']"


## D) README

## E)  Docker commands

## F) Docker File

In [ ]:
FROM python

RUN apt-get update && \
    apt-get install -y rabbitmq-server &&\
    apt install -y python3-pip && \
    pip3 install tweepy  && \ 
    pip3 install pika  && \
    pip3 install nltk

RUN python3 -m nltk.downloader -d /usr/local/share/nltk_data stopwords punkt
RUN mkdir -p /home/application

COPY tweet_processor.py /home/application
COPY tweet_streamer.py /home/application

RUN mkdir -p /home/tweet

VOLUME ["/home/tweet"]

CMD service rabbitmq-server start; exec python3 /home/application/tweet_processor.py & exec python3 /home/application/tweet_streamer.py